# Running SGP4 to calculate and plot Fermi beta angle

### <span style="color:red; font-family:Georgia;">Robert Cameron, July 2015</span>

Based on the python implementation of SGP4 orbit propagator
available at https://pypi.python.org/pypi/sgp4/

In [2]:
import sys
from subprocess import Popen, PIPE, STDOUT
from math import radians, sin, cos, sqrt, acos, asin, degrees
import operator
sys.path.append('/Users/rc/Documents/GLAST/sgp4-1.4')
from sgp4.earth_gravity import wgs84
from sgp4.earth_gravity import wgs84
from sgp4.io import twoline2rv
from sgp4.ext import jday, invjday, days2mdhms

In [3]:
tle = !tail -2 /Users/rc/Documents/GLAST/FGST-TLE.txt

In [4]:
tle

['1 33053U 08029A   15207.89213919  .00000844  00000-0  33035-4 0  9994',
 '2 33053  25.5834  25.8114 0012665 208.6632 151.3207 15.10019820392674']

In [5]:
satellite = twoline2rv(tle[0], tle[1], wgs84)

In [6]:
def crossprod(a, b):
    c = [a[1]*b[2] - a[2]*b[1],
         a[2]*b[0] - a[0]*b[2],
         a[0]*b[1] - a[1]*b[0]]
    return c

In [7]:
ws = tle[0].split()
ydoy = float(ws[3])
year = int(ydoy)/1000 + 2000
jd0 = jday(year,1,1,0,0,0)
ylen = 365
if year % 4 == 0:
    ylen = 366
fypl = []
bpl = []
for doy in range(ylen):
    for hour in range(24):
# add a small amount to the day number to ensure it is always a fraction of a second past the hour
        fdoy = doy + hour/24.0 + 1.0e-9
        jd = jd0 + fdoy
        (y,mon,d,h,m,s) = invjday(jd)
        s = int(s)
        fy = y + fdoy/ylen
        fypl.append(fy)
        pos, vel = satellite.propagate(y,mon,d,h,m,s)
        pole = crossprod(pos,vel)
        polelen = sqrt(pole[0]*pole[0]+pole[1]*pole[1]+pole[2]*pole[2])
# calculate sun coordinates
        n = jd - 2451545.0
        L = (280.460 + 0.9856474 * n) % 360
        g = radians((357.528 + 0.9856003 * n) % 360)
        lamda = radians(L + 1.915 * sin(g) + 0.020 * sin(2*g))
        epsilon = radians(23.439 - 0.0000004 * n)
        Xsun = cos(lamda)
        Ysun = cos(epsilon) * sin(lamda)
        Zsun = sin(epsilon) * sin(lamda)
        dotprod = reduce( operator.add, map( operator.mul, (Xsun,Ysun,Zsun), pole))
        beta = degrees(asin(dotprod/polelen))
        bpl.append(beta)
#        print oformat % (y, mon, d, h, m, s, fy, pos[0], pos[1], pos[2], beta)

NameError: name 'reduce' is not defined